In [3]:
import numpy as np
import pandas as pd
import yaml, glob
import evaluate
import plotly.express as px
import plotly.graph_objects as go

/scratch/work/lunt1/.conda_envs/w2v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [4]:
wer = evaluate.load("wer")

In [76]:
experiment_1 = {
    "BASE": glob.glob("experiments/ex0_base/finnish_ASR_transcrip_fold*.csv"), 
    "BASE_same": glob.glob("experiments/ex0_duplicate/finnish_ASR_transcrip_fold*.csv"),
    
    "3. Additive noise": glob.glob("experiments/ex1_additive_noise/finnish_ASR_transcrip_fold*.csv"),
    "2. Frequency masking": glob.glob("experiments/ex1_band_reject/finnish_ASR_transcrip_fold*.csv"),
    "5. Pitch shift": glob.glob("experiments/ex1_pitch_shift/finnish_ASR_transcrip_fold*.csv"),
    "4. Reverberation": glob.glob("experiments/ex1_reverberation/finnish_ASR_transcrip_fold*.csv"),
    "6. Tempo perturbation": glob.glob("experiments/ex1_tempo_perturbation/finnish_ASR_transcrip_fold*.csv"), 
    "1. Time masking": glob.glob("experiments/ex1_time_masking/finnish_ASR_transcrip_fold*.csv"), 
    "7. Random transformation": glob.glob("experiments/ex1a_random_transform/finnish_ASR_transcrip_fold*.csv")
}

In [77]:
experiment_2 = {
    "BASE": glob.glob("experiments/ex0_base/finnish_ASR_transcrip_fold*.csv"), 
    "BASE_same": glob.glob("experiments/ex0_duplicate/finnish_ASR_transcrip_fold*.csv"),
    
    "Oversampling + augment": glob.glob("experiments/ex2_resample_data_cefr/finnish_ASR_transcrip_fold*.csv"),
}

In [78]:
def get_aggregated_df(paths):
    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        df = df.drop(["labels", "input_values", 
                      "speech", "sampling_rate", "__index_level_0__"], axis=1)
        dfs.append(df)
        
    return pd.concat(dfs).reset_index(drop=True)

In [79]:
def get_wer(group:pd.DataFrame):
    group = group.reset_index()
    stud_wer = 100*wer.compute(predictions=group.ASR_transcript, references=group.text)
    
    # number of words of all utterance by this student
    n = len(" ".join(group.text.tolist()).split()) 
    
    # edit distance
    e = np.round(n*stud_wer)
    
    return  n, e

In [80]:
def asr_statistical_significace(X, rounding=False):
    """
    Input:
        X: pd.DataFrame, with 1) num of words and 2) num of errors columns
    Output:
        WER
    """
    wer_bs = []
    for _ in range(2000):
        n = np.random.choice(X.index, size=len(X), replace=True)
        X_b = X.iloc[n]
        wer_b = X_b.e.sum() / X_b.n.sum()
        wer_bs.append(wer_b)
    
    wer_mean = np.array(wer_bs).mean()
    wer_mean = np.round(wer_mean, 2) if rounding else wer_mean
    return wer_mean

In [119]:
def get_stat_results(exp_results):
    wer_boots = []

    for name, paths in exp_results.items():
        df = get_aggregated_df(paths)

        # obtain df with 1) number of wods and 2) number of errors by student 
        X = df.groupby('student').apply(get_wer)
        X = X.to_frame(name="stat").reset_index()
        X['n'], X['e'] = zip(*X['stat'])

        # Obtain statistically significan wer
        wer_boot = asr_statistical_significace(X)
        wer_boots.append([name, wer_boot])

    df_wer = pd.DataFrame(wer_boots, columns=["Experiment","WER"])
    return df_wer

In [123]:
def plot_exp_results(df_wer, exp_name, up=[], down=[]):
    df_wer['x'] = exp_name
    
    annotations = []

    for _, row in df_wer.iterrows():
        annotations.append({
            "x": row.x, 
            "y": row.WER,
            "xref": "x", 
            "yref": "y",
            "xanchor":"left",
            "text": row.Experiment,
            "showarrow": True,
            "ax": 20,
            "ay": 0,
            "font": {"size":12},
            "xshift":5, 
            "yshift":0
        })

    for i, a in enumerate(annotations):
        if a['text'] in up:
            annotations[i]['ay'] = -10
            annotations[i]['yshift'] = 2
            annotations[i]['xshift'] = 5
        elif a['text'] in down:
            annotations[i]['ay'] = 10
            annotations[i]['yshift'] = -2
            annotations[i]['xshift'] = 5

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_wer.x, 
                   y=df_wer.WER, 
                   mode='markers', 
                   marker=dict(color=px.colors.qualitative.Plotly, size=10)))

    fig.update_layout(plot_bgcolor='whitesmoke', 
                      width=400, 
                      height=600, 
                      annotations=annotations, 
                      xaxis=dict(range=[-1, 7], showticklabels=False))
    fig.show()

In [124]:
df_wer = get_stat_results(experiment_1)
df_wer

,Experiment,WER
0,BASE,21.221064
1,BASE_same,21.221129
2,3. Additive noise,21.159936
3,2. Frequency masking,20.969956
4,5. Pitch shift,21.031450
5,4. Reverberation,20.987033
6,6. Tempo perturbation,21.025107
7,1. Time masking,21.139366
8,7. Random transformation,21.129439


In [125]:
plot_exp_results(df_wer, "Experiment 1")

In [126]:
df_wer_2 = get_stat_results(experiment_2)
df_wer_2

,Experiment,WER
0,BASE,21.240683
1,BASE_same,21.213086
2,Oversampling + augment,21.409521


In [127]:
plot_exp_results(df_wer_2, "Experiment 2")